# Furniture Dataset - Exploratory Data Analysis (EDA)

This notebook explores and analyzes the furniture dataset to understand:
- Dataset structure and quality
- Missing values and data cleaning requirements
- Category distributions for furniture types
- Price analysis and patterns
- Material, color, and brand insights
- Image URL availability and validation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## 1. Load and Initial Data Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('../data/intern_data_ikarus.csv')

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst 3 rows:")
df.head(3)

In [ ]:
# Basic information about the dataset
print("Dataset Info:")
df.info()

print("\n" + "="*50)
print("Missing Values:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percent
})
print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False))

## 2. Data Cleaning and Preprocessing

In [ ]:
# Function to safely parse string representations of lists
def safe_parse_list(val):
    if pd.isna(val) or val == '':
        return []
    try:
        if isinstance(val, str):
            # Clean and parse the string
            val = val.strip()
            if val.startswith('[') and val.endswith(']'):
                return ast.literal_eval(val)
            else:
                # Handle comma-separated values
                return [item.strip().strip('"').strip("'") for item in val.split(',')]
        return val
    except:
        return [str(val)]  # Return as single-item list if parsing fails

# Function to clean price data
def clean_price(price_str):
    if pd.isna(price_str) or price_str == '':
        return np.nan
    try:
        # Remove $ sign and commas, convert to float
        price_cleaned = re.sub(r'[^\d.]', '', str(price_str))
        return float(price_cleaned) if price_cleaned else np.nan
    except:
        return np.nan

# Create a copy for cleaning
df_clean = df.copy()

# Clean price column
df_clean['price_numeric'] = df_clean['price'].apply(clean_price)

# Parse categories and images as lists
df_clean['categories_list'] = df_clean['categories'].apply(safe_parse_list)
df_clean['images_list'] = df_clean['images'].apply(safe_parse_list)

# Clean text columns
text_columns = ['title', 'brand', 'description', 'material', 'color']
for col in text_columns:
    df_clean[col] = df_clean[col].astype(str).str.strip()
    df_clean[col] = df_clean[col].replace(['nan', 'None', ''], np.nan)

print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Valid prices: {df_clean['price_numeric'].notna().sum()} / {len(df_clean)}")
print(f"Products with categories: {df_clean['categories_list'].apply(lambda x: len(x) > 0).sum()}")
print(f"Products with images: {df_clean['images_list'].apply(lambda x: len(x) > 0).sum()}")

## 3. Price Analysis

In [ ]:
# Price statistics
price_stats = df_clean['price_numeric'].describe()
print("Price Statistics:")
print(price_stats)

# Create price visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Furniture Price Analysis', fontsize=16, fontweight='bold')

# Histogram of prices
axes[0, 0].hist(df_clean['price_numeric'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Price Distribution')
axes[0, 0].set_xlabel('Price ($)')
axes[0, 0].set_ylabel('Frequency')

# Box plot of prices
axes[0, 1].boxplot(df_clean['price_numeric'].dropna(), vert=True)
axes[0, 1].set_title('Price Box Plot')
axes[0, 1].set_ylabel('Price ($)')

# Log scale histogram (for better visualization if prices vary widely)
valid_prices = df_clean['price_numeric'].dropna()
valid_prices = valid_prices[valid_prices > 0]
axes[1, 0].hist(np.log10(valid_prices), bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Price Distribution (Log Scale)')
axes[1, 0].set_xlabel('Log10(Price)')
axes[1, 0].set_ylabel('Frequency')

# Price ranges
price_ranges = pd.cut(df_clean['price_numeric'], 
                     bins=[0, 25, 50, 100, 200, 500, float('inf')], 
                     labels=['$0-25', '$25-50', '$50-100', '$100-200', '$200-500', '$500+'])
price_range_counts = price_ranges.value_counts().sort_index()

axes[1, 1].bar(range(len(price_range_counts)), price_range_counts.values, color='coral')
axes[1, 1].set_title('Products by Price Range')
axes[1, 1].set_xlabel('Price Range')
axes[1, 1].set_ylabel('Number of Products')
axes[1, 1].set_xticks(range(len(price_range_counts)))
axes[1, 1].set_xticklabels(price_range_counts.index, rotation=45)

plt.tight_layout()
plt.show()

# Price range distribution table
print("\nPrice Range Distribution:")
for range_label, count in price_range_counts.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{range_label}: {count} products ({percentage:.1f}%)")

## 4. Category Analysis

In [ ]:
# Flatten all categories
all_categories = []
for cat_list in df_clean['categories_list']:
    if isinstance(cat_list, list):
        all_categories.extend([cat.strip() for cat in cat_list if cat.strip()])

category_counts = Counter(all_categories)
top_categories = dict(category_counts.most_common(20))

print(f"Total unique categories: {len(category_counts)}")
print(f"Top 20 categories:")
for category, count in top_categories.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{category}: {count} ({percentage:.1f}%)")

# Visualize top categories
fig, axes = plt.subplots(2, 1, figsize=(15, 12))
fig.suptitle('Furniture Category Analysis', fontsize=16, fontweight='bold')

# Top categories bar chart
categories = list(top_categories.keys())[:15]
counts = list(top_categories.values())[:15]

axes[0].barh(range(len(categories)), counts, color='lightblue')
axes[0].set_title('Top 15 Furniture Categories')
axes[0].set_xlabel('Number of Products')
axes[0].set_yticks(range(len(categories)))
axes[0].set_yticklabels([cat[:30] + '...' if len(cat) > 30 else cat for cat in categories])

# Furniture-specific categories (filter for main furniture types)
furniture_keywords = ['chairs', 'tables', 'furniture', 'storage', 'bedroom', 'living room', 
                     'dining', 'office', 'ottomans', 'barstools', 'nightstands', 'bookcases']

furniture_categories = {}
for category, count in category_counts.items():
    category_lower = category.lower()
    if any(keyword in category_lower for keyword in furniture_keywords):
        furniture_categories[category] = count

# Sort and get top furniture categories
top_furniture = dict(sorted(furniture_categories.items(), key=lambda x: x[1], reverse=True)[:12])

if top_furniture:
    furn_cats = list(top_furniture.keys())
    furn_counts = list(top_furniture.values())
    
    axes[1].pie(furn_counts, labels=[cat[:20] + '...' if len(cat) > 20 else cat for cat in furn_cats], 
                autopct='%1.1f%%', startangle=90)
    axes[1].set_title('Main Furniture Categories Distribution')

plt.tight_layout()
plt.show()

## 5. Brand and Manufacturer Analysis

In [ ]:
# Brand analysis
brand_counts = df_clean['brand'].value_counts().head(15)
manufacturer_counts = df_clean['manufacturer'].value_counts().head(15)

print("Top 15 Brands:")
for brand, count in brand_counts.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{brand}: {count} ({percentage:.1f}%)")

print("\nTop 15 Manufacturers:")
for manufacturer, count in manufacturer_counts.items():
    if pd.notna(manufacturer) and manufacturer != 'nan':
        percentage = (count / len(df_clean)) * 100
        print(f"{manufacturer}: {count} ({percentage:.1f}%)")

# Visualize brands and manufacturers
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('Brand and Manufacturer Analysis', fontsize=16, fontweight='bold')

# Top brands
axes[0].barh(range(len(brand_counts)), brand_counts.values, color='lightcoral')
axes[0].set_title('Top 15 Brands')
axes[0].set_xlabel('Number of Products')
axes[0].set_yticks(range(len(brand_counts)))
axes[0].set_yticklabels(brand_counts.index)

# Top manufacturers (excluding NaN)
valid_manufacturers = manufacturer_counts.dropna()
valid_manufacturers = valid_manufacturers[valid_manufacturers.index != 'nan']
if len(valid_manufacturers) > 0:
    axes[1].barh(range(len(valid_manufacturers)), valid_manufacturers.values, color='lightgreen')
    axes[1].set_title('Top Manufacturers')
    axes[1].set_xlabel('Number of Products')
    axes[1].set_yticks(range(len(valid_manufacturers)))
    axes[1].set_yticklabels(valid_manufacturers.index)

plt.tight_layout()
plt.show()

## 6. Material and Color Analysis

In [ ]:
# Material analysis
material_counts = df_clean['material'].value_counts().head(15)
color_counts = df_clean['color'].value_counts().head(15)

print("Top 15 Materials:")
for material, count in material_counts.items():
    if pd.notna(material) and material != 'nan':
        percentage = (count / len(df_clean)) * 100
        print(f"{material}: {count} ({percentage:.1f}%)")

print("\nTop 15 Colors:")
for color, count in color_counts.items():
    if pd.notna(color) and color != 'nan':
        percentage = (count / len(df_clean)) * 100
        print(f"{color}: {count} ({percentage:.1f}%)")

# Visualize materials and colors
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Material and Color Analysis', fontsize=16, fontweight='bold')

# Materials bar chart
valid_materials = material_counts.dropna()
valid_materials = valid_materials[valid_materials.index != 'nan'][:10]
if len(valid_materials) > 0:
    axes[0, 0].bar(range(len(valid_materials)), valid_materials.values, color='gold')
    axes[0, 0].set_title('Top 10 Materials')
    axes[0, 0].set_xlabel('Material')
    axes[0, 0].set_ylabel('Number of Products')
    axes[0, 0].set_xticks(range(len(valid_materials)))
    axes[0, 0].set_xticklabels(valid_materials.index, rotation=45, ha='right')

# Colors bar chart
valid_colors = color_counts.dropna()
valid_colors = valid_colors[valid_colors.index != 'nan'][:10]
if len(valid_colors) > 0:
    axes[0, 1].bar(range(len(valid_colors)), valid_colors.values, color='lightblue')
    axes[0, 1].set_title('Top 10 Colors')
    axes[0, 1].set_xlabel('Color')
    axes[0, 1].set_ylabel('Number of Products')
    axes[0, 1].set_xticks(range(len(valid_colors)))
    axes[0, 1].set_xticklabels(valid_colors.index, rotation=45, ha='right')

# Material pie chart
if len(valid_materials) > 0:
    axes[1, 0].pie(valid_materials.values, labels=valid_materials.index, autopct='%1.1f%%', startangle=90)
    axes[1, 0].set_title('Material Distribution')

# Color pie chart
if len(valid_colors) > 0:
    axes[1, 1].pie(valid_colors.values, labels=valid_colors.index, autopct='%1.1f%%', startangle=90)
    axes[1, 1].set_title('Color Distribution')

plt.tight_layout()
plt.show()

## 7. Geographic and Dimensional Analysis

In [ ]:
# Country of origin analysis
country_counts = df_clean['country_of_origin'].value_counts().head(10)

print("Top 10 Countries of Origin:")
for country, count in country_counts.items():
    if pd.notna(country) and country != 'nan':
        percentage = (count / len(df_clean)) * 100
        print(f"{country}: {count} ({percentage:.1f}%)")

# Package dimensions analysis
print(f"\nProducts with package dimensions: {df_clean['package_dimensions'].notna().sum()} / {len(df_clean)}")
print(f"Percentage with dimensions: {(df_clean['package_dimensions'].notna().sum() / len(df_clean)) * 100:.1f}%")

# Visualize country distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Geographic and Dimensional Analysis', fontsize=16, fontweight='bold')

# Country of origin
valid_countries = country_counts.dropna()
valid_countries = valid_countries[valid_countries.index != 'nan']
if len(valid_countries) > 0:
    axes[0].pie(valid_countries.values, labels=valid_countries.index, autopct='%1.1f%%', startangle=90)
    axes[0].set_title('Products by Country of Origin')

# Data completeness by field
completeness_data = {
    'Title': df_clean['title'].notna().sum(),
    'Price': df_clean['price_numeric'].notna().sum(),
    'Category': df_clean['categories_list'].apply(lambda x: len(x) > 0).sum(),
    'Images': df_clean['images_list'].apply(lambda x: len(x) > 0).sum(),
    'Brand': df_clean['brand'].notna().sum(),
    'Material': (df_clean['material'].notna() & (df_clean['material'] != 'nan')).sum(),
    'Color': (df_clean['color'].notna() & (df_clean['color'] != 'nan')).sum(),
    'Description': df_clean['description'].notna().sum()
}

fields = list(completeness_data.keys())
percentages = [(count / len(df_clean)) * 100 for count in completeness_data.values()]

axes[1].bar(fields, percentages, color='skyblue')
axes[1].set_title('Data Completeness by Field')
axes[1].set_xlabel('Fields')
axes[1].set_ylabel('Completeness (%)')
axes[1].set_xticklabels(fields, rotation=45, ha='right')
axes[1].set_ylim(0, 100)

# Add percentage labels on bars
for i, v in enumerate(percentages):
    axes[1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 8. Image URL Analysis

In [ ]:
# Analyze image URLs
image_stats = {
    'products_with_images': 0,
    'total_images': 0,
    'avg_images_per_product': 0,
    'amazon_urls': 0,
    'valid_urls': 0
}

all_images = []
products_with_images = 0
amazon_url_count = 0

for images in df_clean['images_list']:
    if isinstance(images, list) and len(images) > 0:
        # Filter out empty strings
        valid_images = [img.strip() for img in images if img.strip()]
        if valid_images:
            products_with_images += 1
            all_images.extend(valid_images)
            
            # Count Amazon URLs
            amazon_urls = [img for img in valid_images if 'amazon' in img.lower()]
            amazon_url_count += len(amazon_urls)

image_stats['products_with_images'] = products_with_images
image_stats['total_images'] = len(all_images)
image_stats['avg_images_per_product'] = len(all_images) / products_with_images if products_with_images > 0 else 0
image_stats['amazon_urls'] = amazon_url_count
image_stats['valid_urls'] = len([img for img in all_images if img.startswith('http')])

print("Image URL Statistics:")
print(f"Products with images: {image_stats['products_with_images']} / {len(df_clean)} ({(image_stats['products_with_images']/len(df_clean))*100:.1f}%)")
print(f"Total image URLs: {image_stats['total_images']}")
print(f"Average images per product: {image_stats['avg_images_per_product']:.1f}")
print(f"Amazon URLs: {image_stats['amazon_urls']} ({(image_stats['amazon_urls']/image_stats['total_images'])*100:.1f}%)")
print(f"Valid HTTP URLs: {image_stats['valid_urls']} ({(image_stats['valid_urls']/image_stats['total_images'])*100:.1f}%)")

# Sample image URLs
print("\nSample image URLs:")
sample_images = all_images[:5] if all_images else []
for i, img in enumerate(sample_images, 1):
    print(f"{i}. {img[:100]}..." if len(img) > 100 else f"{i}. {img}")

## 9. Text Analysis for AI Model Preparation

In [ ]:
# Text length analysis for title and description
df_clean['title_length'] = df_clean['title'].astype(str).str.len()
df_clean['description_length'] = df_clean['description'].astype(str).str.len()

# Word count analysis
df_clean['title_word_count'] = df_clean['title'].astype(str).str.split().str.len()
df_clean['description_word_count'] = df_clean['description'].astype(str).str.split().str.len()

text_stats = {
    'avg_title_length': df_clean['title_length'].mean(),
    'avg_description_length': df_clean['description_length'].mean(),
    'avg_title_words': df_clean['title_word_count'].mean(),
    'avg_description_words': df_clean['description_word_count'].mean()
}

print("Text Statistics for AI Model Training:")
print(f"Average title length: {text_stats['avg_title_length']:.1f} characters")
print(f"Average description length: {text_stats['avg_description_length']:.1f} characters")
print(f"Average title word count: {text_stats['avg_title_words']:.1f} words")
print(f"Average description word count: {text_stats['avg_description_words']:.1f} words")

# Create combined text for embeddings
def create_combined_text(row):
    """Combine title, description, categories, material, and color for embeddings"""
    text_parts = []
    
    if pd.notna(row['title']) and row['title'] != 'nan':
        text_parts.append(str(row['title']))
    
    if pd.notna(row['description']) and row['description'] != 'nan':
        text_parts.append(str(row['description']))
    
    if isinstance(row['categories_list'], list) and len(row['categories_list']) > 0:
        text_parts.append(' '.join(row['categories_list']))
    
    if pd.notna(row['material']) and row['material'] != 'nan':
        text_parts.append(f"material: {row['material']}")
    
    if pd.notna(row['color']) and row['color'] != 'nan':
        text_parts.append(f"color: {row['color']}")
    
    return ' '.join(text_parts)

df_clean['combined_text'] = df_clean.apply(create_combined_text, axis=1)

print(f"\nAverage combined text length: {df_clean['combined_text'].str.len().mean():.1f} characters")
print(f"Average combined text word count: {df_clean['combined_text'].str.split().str.len().mean():.1f} words")

# Sample combined texts
print("\nSample combined texts for embedding:")
for i in range(3):
    sample_text = df_clean['combined_text'].iloc[i]
    print(f"\n{i+1}. {sample_text[:200]}..." if len(sample_text) > 200 else f"\n{i+1}. {sample_text}")

## 10. Data Quality Summary and Recommendations

In [ ]:
# Generate comprehensive data quality report
print("=" * 60)
print("DATA QUALITY SUMMARY & RECOMMENDATIONS")
print("=" * 60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"• Total products: {len(df_clean)}")
print(f"• Unique product IDs: {df_clean['uniq_id'].nunique()}")
print(f"• Duplicate products: {len(df_clean) - df_clean['uniq_id'].nunique()}")

print(f"\n💰 PRICING DATA:")
valid_prices = df_clean['price_numeric'].notna().sum()
print(f"• Products with valid prices: {valid_prices} / {len(df_clean)} ({(valid_prices/len(df_clean))*100:.1f}%)")
if valid_prices > 0:
    print(f"• Price range: ${df_clean['price_numeric'].min():.2f} - ${df_clean['price_numeric'].max():.2f}")
    print(f"• Median price: ${df_clean['price_numeric'].median():.2f}")

print(f"\n🏷️ CATEGORY DATA:")
products_with_cats = df_clean['categories_list'].apply(lambda x: len(x) > 0).sum()
print(f"• Products with categories: {products_with_cats} / {len(df_clean)} ({(products_with_cats/len(df_clean))*100:.1f}%)")
print(f"• Total unique categories: {len(category_counts)}")
print(f"• Most common category: {list(category_counts.keys())[0]} ({list(category_counts.values())[0]} products)")

print(f"\n🖼️ IMAGE DATA:")
print(f"• Products with images: {image_stats['products_with_images']} / {len(df_clean)} ({(image_stats['products_with_images']/len(df_clean))*100:.1f}%)")
print(f"• Total image URLs: {image_stats['total_images']}")
print(f"• Average images per product: {image_stats['avg_images_per_product']:.1f}")

print(f"\n📝 TEXT DATA:")
titles_available = (df_clean['title'].notna() & (df_clean['title'] != 'nan')).sum()
descriptions_available = (df_clean['description'].notna() & (df_clean['description'] != 'nan')).sum()
print(f"• Products with titles: {titles_available} / {len(df_clean)} ({(titles_available/len(df_clean))*100:.1f}%)")
print(f"• Products with descriptions: {descriptions_available} / {len(df_clean)} ({(descriptions_available/len(df_clean))*100:.1f}%)")

print(f"\n🔧 DATA CLEANING RECOMMENDATIONS:")
print(f"• ✅ Price cleaning: Remove $ symbols, convert to numeric")
print(f"• ✅ Category parsing: Convert string representations to lists")
print(f"• ✅ Image URL validation: Check for valid HTTP URLs")
print(f"• ✅ Text normalization: Clean and standardize text fields")
print(f"• ✅ Missing value imputation: Use category medians for prices")
print(f"• ✅ Duplicate removal: Based on unique_id field")

print(f"\n🤖 AI MODEL READINESS:")
print(f"• ✅ Text embedding: {len(df_clean)} products have combined text")
print(f"• ✅ Image processing: {image_stats['products_with_images']} products have images")
print(f"• ✅ Category classification: {len(category_counts)} unique categories")
print(f"• ✅ Price prediction: {valid_prices} products with pricing data")

print(f"\n💡 BUSINESS INSIGHTS:")
if valid_countries := country_counts.dropna():
    top_country = valid_countries.index[0]
    print(f"• Most products manufactured in: {top_country}")
if valid_materials := material_counts.dropna():
    top_material = valid_materials.index[0]
    print(f"• Most common material: {top_material}")
if valid_colors := color_counts.dropna():
    top_color = valid_colors.index[0]
    print(f"• Most popular color: {top_color}")
print(f"• Price range diversity: {len(price_range_counts)} price segments")

# Save cleaned dataset
df_clean.to_csv('../data/cleaned_furniture_data.csv', index=False)
print(f"\n💾 Cleaned dataset saved as: cleaned_furniture_data.csv")
print(f"Ready for AI model training and application development!")